In [ ]:
upstream = ['01-uniform-files']
product = None

In [3]:
import pandas as pd
import multiprocessing as mp
import subprocess

In [4]:
reference = pd.read_csv(upstream['01-uniform-files']['data'], index_col=None)

In [71]:

def run_us_align(x, y, z):
    us_align_out = subprocess.run(f"USalign -mm 1 -ter 1 -outfmt 2 pdb/{x}.pdb pdb/{y}.pdb", capture_output=True, shell=True).stdout.decode()
    us_align_data = us_align_out.split('\n')[1]
    us_align_data = us_align_data.split()
    out = dict(
        pdb1=us_align_data[0].split('.pdb')[0].replace('pdb/', ''),
        pdb2=us_align_data[1].split('.pdb')[0].replace('pdb/', ''),
        chain1=us_align_data[0].split('.pdb')[1],
        chain2=us_align_data[1].split('.pdb')[1],
        tm1=float(us_align_data[2]),
        tm2=float(us_align_data[3]),
        rmsd=float(us_align_data[4]),
        id1=float(us_align_data[5]),
        id2=float(us_align_data[6]),
        idali=float(us_align_data[7]),
        l1=int(us_align_data[8]),
        l2=int(us_align_data[9]),
        lali=int(us_align_data[10]),
        variant=z
    )

    return out

In [11]:
ancestors = reference.query('is_ancestral == True').copy()
ancestors['lineage'] = ancestors['lineage'].apply(lambda x: '_'.join(x.split('_')[:-1]))
lineage = ancestors['lineage'].unique()
ancestors = ancestors[ancestors['lineage'].isin(lineage)]

,original_name,id,stochiometry,average_pLDDT,taxond_id,lineage,scientific_name,is_ancestral,nitrogenase_type,status,chain_reference,variant,clade
385,Anc_1392_map_001.rechained.pdb,nsdb-000386,DDKK,94.605305,Anc_1392_map,Anc_1392,Anc_1392_map,True,Anc,gold,"{'A': 'D', 'B': 'K', 'D': 'D', 'E': 'K'}",MAP,nif-ii
386,Anc_1414_map_001.rechained.pdb,nsdb-000387,DDKK,94.751496,Anc_1414_map,Anc_1414,Anc_1414_map,True,Anc,gold,"{'A': 'D', 'B': 'K', 'D': 'D', 'E': 'K'}",MAP,nif-ii
387,Anc_1242_map_001.rechained.pdb,nsdb-000388,DDKK,96.049193,Anc_1242_map,Anc_1242,Anc_1242_map,True,Anc,gold,"{'A': 'D', 'B': 'K', 'D': 'D', 'E': 'K'}",MAP,nif-i
388,Anc_1339_map_001.rechained.pdb,nsdb-000389,DDKK,94.677571,Anc_1339_map,Anc_1339,Anc_1339_map,True,Anc,gold,"{'A': 'D', 'B': 'K', 'D': 'D', 'E': 'K'}",MAP,nif-i
389,Anc_1346_map_001.rechained.pdb,nsdb-000390,DDKK,95.382665,Anc_1346_map,Anc_1346,Anc_1346_map,True,Anc,gold,"{'A': 'D', 'B': 'K', 'D': 'D', 'E': 'K'}",MAP,nif-ii
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,Anc_779_alt3_001.rechained.pdb,nsdb-005364,HH,90.007800,Anc_779_alt3,Anc_779,Anc_779_alt3,True,Anc,silver,"{'A': 'H', 'B': 'H'}",ALT3,other
5364,Anc_1538_alt3_001.rechained.pdb,nsdb-005365,HH,90.627836,Anc_1538_alt3,Anc_1538,Anc_1538_alt3,True,Anc,silver,"{'A': 'H', 'B': 'H'}",ALT3,other
5365,Anc_1486_alt4_001.rechained.pdb,nsdb-005366,HH,87.068425,Anc_1486_alt4,Anc_1486,Anc_1486_alt4,True,Anc,silver,"{'A': 'H', 'B': 'H'}",ALT4,other
5366,Anc_1464_alt2_001.rechained.pdb,nsdb-005367,HH,87.755495,Anc_1464_alt2,Anc_1464,Anc_1464_alt2,True,Anc,silver,"{'A': 'H', 'B': 'H'}",ALT2,other


In [66]:
def process_variant_batch(u):
    u = u.set_index('variant')
    ml = u.loc['MAP']
    #print(u)
    variants = u[u.index.isin(['ALTALL', 'ALT2', 'ALT3', 'ALT4', 'ALT5'])]
    #print(variants)
    group = []
    pool = mp.Pool(5)
    group = [
        pool.apply_async(
            run_us_align, args=(item.id, ml.id)
        ) for _, item in variants.iterrows()
    ]
    group = pd.DataFrame.from_records([p.get() for p in group])
    return group[['rmsd', 'tm1', 'tm2', 'idali']].mean()

In [ ]:
def process_variant_batch2(map_variant, u, out):

    variants = u.query('lineage == "{0}" and variant != "MAP" and stochiometry == "{1}"'.format(map_variant.lineage, map_variant.stochiometry))
    group = []
    pool = mp.Pool(5)
    group = [
        pool.apply_async(
            run_us_align, args=(item.id, map_variant.id, item.variant)
        ) for _, item in variants.iterrows()
    ]
    group = [p.get() for p in group]
    for g in group:
        g['stochiometry'] = map_variant['stochiometry']
        g['lineage'] = map_variant['lineage']
    out += group 
    return None

In [ ]:
out = []
ancestors.query(
    'variant == "MAP"'
).apply(
    lambda x: process_variant_batch2(x, ancestors, out), axis=1
)

In [ ]:
pd.DataFrame.from_records(out).to_json(
    product['data'],
    orient='records', indent=4
)

In [ ]:
# ancestors.query(
#     'variant == ""MAP'
# ).apply(process_variant_batch).to_json(
#     product['data'],
#     orient='records'
# )

In [70]:
# ancestors.groupby(
#     ['lineage', 'stochiometry'], as_index=False
# ).apply(process_variant_batch).to_json(
#     product['data'],
#     orient='records'
# )

/tmp/ipykernel_1762053/1644870084.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ancestors.query('lineage == "Anc_1206"').groupby(['lineage', 'stochiometry'], as_index=False).apply(process_variant_batch).to_json(orient='records')


'[{"lineage":"Anc_1206","stochiometry":"DDKK","rmsd":0.67,"tm1":0.99756,"tm2":0.99756,"idali":0.8674},{"lineage":"Anc_1206","stochiometry":"HH","rmsd":0.22,"tm1":0.99926,"tm2":0.99926,"idali":0.9656}]'